In [3]:
GLOBAL_LOADED_PUZZLES=True

In [4]:
import numpy as np
import abc

In [3]:
class Sudoku:
    def __init__(self,blocksize=3,autoinit=True): #Board size = blocksize*blocksize x blocksize*blocksize
        self.blocksize=blocksize
        maxval=int(blocksize**2)
        self.maxval=maxval
        self.grid=np.zeros((maxval,maxval))
        if autoinit: self.initGrid()
    
    def initGrid(self,iterlim=50,tries=100):
        #print("Tries left",tries)
        if tries<=0: return False
        blocks=[]
        for i in range(0,self.maxval,self.blocksize):
            blocks.append([])
            for j in range(0,self.maxval,self.blocksize):
                ch=np.array(range(self.maxval))+1; np.random.shuffle(ch)
                ch=ch.reshape((self.blocksize,self.blocksize))
                
                k=j//self.blocksize #j//self.blocksize
                #print(i,j,"as focal block coordinates")
                hblks=[]
                for myblock in blocks[-1]: hblks.append(myblock)
                self.solveHorizontal(ch,hblks)
                vblks=[]
                for blkid in range(len(blocks)):
                    if len(blocks[blkid])<=k: continue
                    myblock=blocks[blkid][k]
                    #print("My block",myblock,"with ID",blkid)
                    vblks.append(myblock)
                #print(ch,vblks,k)
                converged=False
                Kit=0
                succ=True
                while not converged:
                    ch,c1=self.solveVertical(ch,vblks)
                    c2=self.solveHorizontal(ch,hblks)
                    converged=(c1 and c2)
                    Kit+=1
                    #print(Kit,"iterations")
                    if Kit>iterlim:
                        succ=False
                        break
                if not succ: return self.initGrid(iterlim,tries-1)
                blocks[-1].append(ch)
                self.grid[i:i+self.blocksize,j:j+self.blocksize]=ch
        return blocks
    
    def solveHorizontal(self,focal,blocks,tag=True):
        if not len(blocks): return True
        #if tag: print("Solve Horizontal")
        swaps=dict()
        swapped=True
        singleswap=False
        while swapped:
            swapped=False
            for rowid in range(self.blocksize):
                for colid in range(self.blocksize):
                    noclash=list(range(self.blocksize)) #Rows where there is no clash
                    for browid in range(self.blocksize):
                        if browid not in noclash: continue
                        for bcolid in range(self.blocksize):
                            for blk in blocks:
                                if blk[browid][bcolid]==focal[rowid][colid]:
                                    noclash.remove(browid)
                                    break
                            if browid not in noclash: break
                    #print(noclash,rowid,rowid not in noclash,"for value",focal[rowid][colid])
                    #print(focal)
                    #print(blocks)
                    if rowid not in noclash:
                        if not len(noclash): raise ValueError("Sorry! Cannot fix this puzzle")
                        excl=np.concatenate([nblk[rowid] for nblk in blocks])
                        #print(excl,"as exclusion with noclash=",noclash)
                        tv=excl[0] # if len(excl) else 0
                        K=0
                        while (tv in excl) and K<self.blocksize*2:
                            nrid=np.random.choice(noclash)
                            ncid=int(np.random.uniform(0,self.blocksize))
                            if ((rowid,colid) in swaps) and (nrid,ncid) in swaps[(rowid,colid)]: continue
                            tv=focal[nrid][ncid]
                            K+=1
                        focal[rowid][colid],focal[nrid][ncid]=focal[nrid][ncid],focal[rowid][colid]
                        #print("\tSwapped:",rowid,colid,"with",nrid,ncid)
                        if (rowid,colid) in swaps:
                            swaps[(rowid,colid)].append((nrid,ncid))
                        else:
                            swaps[(rowid,colid)]=[(nrid,ncid)]
                        swapped=True
                        singleswap=True
            #if not singleswap: print(focal,blocks)
            return (not singleswap) #Converged?
    
    def solveVertical(self,focal,blocks):
        #print("Solve vertical")
        focalX=focal.transpose()
        nblocks=[blk.transpose() for blk in blocks]
        #print(focalX,nblocks)
        ret=self.solveHorizontal(focalX,nblocks,tag=False)
        focal=focalX.transpose()
        return focal,ret
    def __str__(self): return str(self.grid)
    def __rep__(self): return str(self)

In [58]:
#sud=Sudoku()
#print(sud.grid)
#np.sum(sud.grid,axis=1),np.sum(sud.grid,axis=0)

Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
Solve vertical
[[5. 6. 4. 2. 3. 9. 7. 1. 8.]
 [7. 9. 2. 8. 1. 4. 5. 6. 3.]
 [1. 8. 3. 5. 7. 6. 4. 9. 2.]
 [3. 4. 5. 9. 8. 1. 2. 7. 6.]
 [6. 7. 1. 3. 5. 2. 8. 4. 9.]
 [9. 2. 8. 4. 6. 7. 3. 5. 1.]
 [4. 3. 6. 1. 9. 5. 8. 2. 7.]
 [8. 5. 9. 7. 2. 3. 6. 1. 4.]
 [2. 1. 7. 6. 4. 8. 9. 3. 5.]]


(array([45., 45., 45., 45., 45., 45., 45., 45., 45.]),
 array([45., 45., 45., 45., 45., 45., 52., 38., 45.]))